In [2]:
#그룹 연산
import pandas as pd 
df = pd.read_csv("c:\\work\\gapminder.tsv", sep="\t")

In [3]:
#year열을 기준으로 데이터를 그룹화한 다음 lifeExp열의 평균을 구한다.
avg_life_exp_by_year = df.groupby('year').lifeExp.mean()
print(avg_life_exp_by_year)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


In [4]:
#분할-반영-결합 과정 살펴보기 
#실제로 groupby메서드에 life열을 전달하면 가장 먼저 연도별로 데이터를 
#나누는 과정이 진행된다. 다음은 year열의 데이터를 중복없이 추출한 것이다. 
#groupby메서드에 열 이름을 전달하면 이런 '분할'작업이 먼저 일어난다고 이해한다.
years = df.year.unique()
print(years)

[1952 1957 1962 1967 1972 1977 1982 1987 1992 1997 2002 2007]


In [5]:
#그런 다음 연도별로 평균값을 구한다. 
#그러려면 일단 각 연도별로 데이터를 추출해야한다. 
#다음은 1952년의 데이터를 추출한 것이다. 이 과정을 '반영'작업의
#한 부분이라고 이해하면 된다. 
y1952 = df.loc[df.year == 1952, :]
print(y1952.head())

        country continent  year  lifeExp       pop    gdpPercap
0   Afghanistan      Asia  1952   28.801   8425333   779.445314
12      Albania    Europe  1952   55.230   1282697  1601.056136
24      Algeria    Africa  1952   43.077   9279525  2449.008185
36       Angola    Africa  1952   30.015   4232095  3520.610273
48    Argentina  Americas  1952   62.485  17876956  5911.315053


In [6]:
#아직 lifeExp열의 평균값을 구하지 않았따. 
#다음은 앞에서 추출한 1952년의 데이터에서 lifeExp열의 평균값을 구한 것이다. 
#이 과정도 '반영'작업의 한 부분이다.
y1952_mean = y1952.lifeExp.mean()
print(y1952_mean)

49.057619718309866


In [11]:
#앞의 과정을 반복하여 남은 연도의 평균값을 구하면 비로소 '반영'작업이 끝난다.
y1957 = df.loc[df.year == 1957, :]
y1957_mean = y1957.lifeExp.mean() 
print(y1957_mean)

y2007 = df.loc[df.year == 2007, :]
y2007_mean = y2007.lifeExp.mean() 
print(y2007_mean)


51.50740112676056
67.00742253521126


In [12]:
#마지막으로 연도별로 계산한 lifeExp의 평균값을 합친다. 
#바로 이 과정이 '결합'과정이다. 
df2 = pd.DataFrame({'year':[1952,1952,2007],
    '':[y1952_mean,y1957_mean,y2007_mean]})
print(df2)

   year           
0  1952  49.057620
1  1952  51.507401
2  2007  67.007423


In [13]:
#다음은 입력받은 열의 평균값을 구하는 함수이다.
def my_mean(values):
    n = len(values)
    sum = 0
    for value in values:
        sum += value 
    return sum/n


In [14]:
#앞에서 만든 함수를 groupby메서드와 조합하기 위해
#agg메서드를 사용한다. 결과를 보면 mean메서드를 사용하여 얻은 값과
#동일하다는 것을 알 수있다.
agg_my_mean = df.groupby('year').lifeExp.agg(my_mean)
print(agg_my_mean)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


In [15]:
#이번에는 2개의 인자값을 받아 처리하는 사용자정의함수를 만들어본다.
#다음은 첫번째 인자로 받은 열의 평균값을 구하여 두번째 인자로 받은 값과의
#차이를 계산한 다음 반환하는 함수이다.
def my_mean_diff(values, diff_value):
    n = len(values)
    sum = 0
    for value in values:
        sum += value 
    mean = sum / n 
    return mean - diff_value

In [16]:
#다음은 연도별 평균 수명에서 전체 평균 수명을 뺀 값을 구한 것이다.
#agg메서드의 첫번째 인자에 my_mean_diff함수를 전달하고 두번째 인자에 
#전체 평균 수명값을 전달했다.
global_mean = df.lifeExp.mean()
print(global_mean)

agg_mean_diff = df.groupby('year').lifeExp.agg(my_mean_diff, diff_value=global_mean)
print(agg_mean_diff)

59.474439366197174
year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64


In [17]:
#여러개의 집계 메서드를 한번에 사용하기
#집계 메서드를 리스트나 딕셔너리에 담아 agg메서드에 
#전달하면 된다.
#다음은 연도별로 그룹화한 lifeExp열의 0이 아닌 값의 개수, 평균, 표준편차를
#한번에 계산하여 출력한 것이다. 넘파이 메서드인 count_nonzero, mean, std를
#리스트에 담아 agg메서드에 전달한다.
import numpy as np
gdf = df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])
print(gdf)

      count_nonzero       mean        std
year                                     
1952            142  49.057620  12.225956
1957            142  51.507401  12.231286
1962            142  53.609249  12.097245
1967            142  55.678290  11.718858
1972            142  57.647386  11.381953
1977            142  59.570157  11.227229
1982            142  61.533197  10.770618
1987            142  63.212613  10.556285
1992            142  64.160338  11.227380
1997            142  65.014676  11.559439
2002            142  65.694923  12.279823
2007            142  67.007423  12.073021


In [18]:
#이번에는 집계 메서드를 딕셔너리에 담아 agg메서드에 전달해 본다. 
gdf_dict = df.groupby('year').agg({'lifeExp':'mean','pop':'median',
    'gdpPercap':'median'})
print(gdf_dict)

        lifeExp         pop    gdpPercap
year                                    
1952  49.057620   3943953.0  1968.528344
1957  51.507401   4282942.0  2173.220291
1962  53.609249   4686039.5  2335.439533
1967  55.678290   5170175.5  2678.334740
1972  57.647386   5877996.5  3339.129407
1977  59.570157   6404036.5  3798.609244
1982  61.533197   7007320.0  4216.228428
1987  63.212613   7774861.5  4280.300366
1992  64.160338   8688686.5  4386.085502
1997  65.014676   9735063.5  4781.825478
2002  65.694923  10372918.5  5319.804524
2007  67.007423  10517531.0  6124.371108


In [20]:
#데이터 변환
#표준점수를 계산하는 함수
def my_zscore(x):
    return (x - x.mean()) / x.std()

#다음은 각 연도별 lifeExp열의 표준점수를 계산한 것이다. my_zcore함수를 적용하기 
#위해 transform메서드를 사용했다.
transform_z = df.groupby('year').lifeExp.transform(my_zscore)
print(transform_z.head())

0   -1.656854
1   -1.731249
2   -1.786543
3   -1.848157
4   -1.894173
Name: lifeExp, dtype: float64


In [21]:
print(df.shape)

print(transform_z.shape)

(1704, 6)
(1704,)


In [22]:
#누락값을 평균으로 처리하기 - 가끔은 누락값을 평균값으로 처리하는 것이 더 좋을때가 있다.
#다음은 seaborn라이브러리의 tips데이터집합에서 1개의 행 데이터만 가져온 다음
#total_bill열의 값 4개를 임의로 선택하여 누락값으로 바꾼 것이다.
import seaborn as sns
import numpy as np

np.random.seed(42)
tips_10 = sns.load_dataset('tips').sample(10)
tips_10.loc[np.random.permutation(tips_10.index)[:4], 'total_bill'] = np.NaN
print(tips_10)


     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153         NaN  2.00    Male     No   Sun  Dinner     4
211         NaN  5.16    Male    Yes   Sat  Dinner     4
198         NaN  2.00  Female    Yes  Thur   Lunch     2
176         NaN  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [23]:
#그런데 total_bill열의 누락값을 단순히 total_bill열의 평균값으로 채우면 안된다.
#현재 tips_10이 데이터는 여성보다 남성이 더 많다. 즉 여성과 남성을 구분하여
#total_bill열의 평균값을 구해야 한다. 
count_sex = tips_10.groupby('sex').count()
print(count_sex)

        total_bill  tip  smoker  day  time  size
sex                                             
Male             4    7       7    7     7     7
Female           2    3       3    3     3     3


In [24]:
#다음은 성별을 구분하여 total_bill열의 데이터를 받아 평균값을 구하는 함수다.
def fill_na_mean(x):
    avg = x.mean()
    return x.fillna(avg)

In [25]:
#다음은 성별을 구분한 total_bill열의 데이터를 fill_na_mean함수에 전달하여
#평균값을 구한 다음 tips_10에 새로운 열로 추가한 것이다. 
#남성과 여성의 누락값을 고려하여 계산한 평균값으로 잘 채워져 있는 것을 알 수 있다.
total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)
tips_10['fill_total_bill'] = total_bill_group_mean
print(tips_10)

     total_bill   tip     sex smoker   day    time  size  fill_total_bill
24        19.82  3.18    Male     No   Sat  Dinner     2          19.8200
6          8.77  2.00    Male     No   Sun  Dinner     2           8.7700
153         NaN  2.00    Male     No   Sun  Dinner     4          17.9525
211         NaN  5.16    Male    Yes   Sat  Dinner     4          17.9525
198         NaN  2.00  Female    Yes  Thur   Lunch     2          13.9300
176         NaN  2.00    Male    Yes   Sun  Dinner     2          17.9525
192       28.44  2.56    Male    Yes  Thur   Lunch     2          28.4400
124       12.48  2.52  Female     No  Thur   Lunch     2          12.4800
9         14.78  3.23    Male     No   Sun  Dinner     2          14.7800
101       15.38  3.00  Female    Yes   Fri  Dinner     2          15.3800


In [26]:
#데이터 필터링
tips = sns.load_dataset('tips')
print(tips.shape)

(244, 7)


In [27]:
#size열의 데이터 수를 확인해 보면 1,5,6테이블의 주문이 매우 적다는 것을
#알 수 있다.
print(tips['size'].value_counts())

2    156
3     38
4     37
5      5
1      4
6      4
Name: size, dtype: int64


In [28]:
#상황에 따라 이런 데이터는 제외하기도 한다. 만약 30번 이상의 주문이 있는 
#테이블만 추려 데이터 분석을 하려면 어떻게 해야 할까? 
#다음은 30번 이상의 주문이 있는 테이블만 그룹화하여 변수 tips_filtered에 저장한 것이다.
tips_filtered = tips.groupby('size').filter(lambda x:x['size'].count() >= 30)

In [29]:
print(tips_filtered.shape)

(231, 7)


In [30]:
print(tips_filtered['size'].value_counts())

2    156
3     38
4     37
Name: size, dtype: int64


In [31]:
#그룹 오브젝트
tips_10 = sns.load_dataset('tips').sample(10, random_state=42)
print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153       24.55  2.00    Male     No   Sun  Dinner     4
211       25.89  5.16    Male    Yes   Sat  Dinner     4
198       13.00  2.00  Female    Yes  Thur   Lunch     2
176       17.89  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [32]:
#groupby메서드의 결과값을 출력하면 자료형이 그룹 오브제트라는 것을 확인할 수 있다.
grouped = tips_10.groupby('sex')
print(grouped)

In [33]:
#그룹 오브젝트에 포함된 그룹을 보려면 grouped속성을 출력하면 된다. 
#그러면 sex열로 그룹화한 데이터프레임의 인덱스를 확인할 수 있다. 
print(grouped.groups)

{'Male': [24, 6, 153, 211, 176, 192, 9], 'Female': [198, 124, 101]}


In [34]:
#앞에서 만든 그룹 오브젝트(grouped)을 이용하여 평균을 구해본다. 
#그러면 tips데이터집합의 모든 열의 평균을 구한 것이 아니라 
#total_bill, tip, size열의 평균을 구했다는 것을 알 수 있다. 
avgs = grouped.mean()
print(avgs)

        total_bill       tip      size
sex                                   
Male         20.02  2.875714  2.571429
Female       13.62  2.506667  2.000000


In [35]:
#tips데이터집합의 열을 확인해 보면 평균값을 계산할 수 없는 열인 
#smoker, day, time열은 그룹 연산에서 제외되었다는 것을 알 수 있다. 
#이처럼 파이썬은 그룹 연산에 적합한 열을 알아서 골라준다. 
print(tips_10.columns)


Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')


In [36]:
#만약 그룹오브젝트에서 특정 데이터만 추출하려면 get_group메서드를 사용하면 된다.
#다음은 sex열로 그룹화한 그룹 오브젝트에 get_group메서드를 사용하여 성별이 여성인
#데이터만 추출한 것이다.
female = grouped.get_group('Female')
print(female)

     total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [37]:
#이번에는 그룹 오브젝트를 반복문에 사용해 본다.
#sex열을 기준으로 그룹화한 tips데이터집합은 여성그룹과 남성그룹으로 
#나누어져있다. 이 특징을 이용하녀 반복문을 사용하면 된다. 
for sex_group in grouped:
    print(sex_group)

('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2)
('Female',      total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2)


In [38]:
#여러 열을 사용해 그룹 오브젝트 만들고 계산하기 
#여러 열을 사용하여 데이터를 그룹화하려면 리스트에 열 이름을 담아
#groupby메서드에 전달하면 된다. 
#다음은 sex, time열을 기준으로 데이터를 그룹화하고 평균값을 구한 것이다.
bill_sex_time = tips_10.groupby(['sex','time'])
group_avg = bill_sex_time.mean()
print(group_avg)

               total_bill       tip      size
sex    time                                  
Male   Lunch    28.440000  2.560000  2.000000
       Dinner   18.616667  2.928333  2.666667
Female Lunch    12.740000  2.260000  2.000000
       Dinner   15.380000  3.000000  2.000000


In [40]:
#앞에서 만든 group_avg의 자료형을 확인해보면 데이터프레임이라는 것을 알 수 있다. 
#그리고 변수 group_avg에 포함된 열은 total_bill, tip, size라는 것도 알 수 있다. 
print(type(group_avg))
print(group_avg.columns)

<class 'pandas.core.frame.DataFrame'>
Index(['total_bill', 'tip', 'size'], dtype='object')


In [41]:
#인덱스는 어떻게 구성되어 있을까?
print(group_avg.index)

MultiIndex([(  'Male',  'Lunch'),
            (  'Male', 'Dinner'),
            ('Female',  'Lunch'),
            ('Female', 'Dinner')],
           names=['sex', 'time'])


In [42]:
#위와 같이 데이터프레임의 인덱스가 MultiIndex인 경우
#reset_index메서드를 사용하여 데이터프레임의 인덱스를 새로 부여할 수 있다. 
group_method = tips_10.groupby(['sex','time']).mean().reset_index()
print(group_method)

      sex    time  total_bill       tip      size
0    Male   Lunch   28.440000  2.560000  2.000000
1    Male  Dinner   18.616667  2.928333  2.666667
2  Female   Lunch   12.740000  2.260000  2.000000
3  Female  Dinner   15.380000  3.000000  2.000000


In [45]:
#reset_index메서드 대신 as_index인자를 False로 설정해도 앞의 과정처럼 결과를
#얻을 수 있다.
group_param = tips_10.groupby(['sex','time'], as_index=False).mean()
print(group_param)

      sex    time  total_bill       tip      size
0    Male   Lunch   28.440000  2.560000  2.000000
1    Male  Dinner   18.616667  2.928333  2.666667
2  Female   Lunch   12.740000  2.260000  2.000000
3  Female  Dinner   15.380000  3.000000  2.000000
